# Plotting

This chapter is utterly incomplete and probably not worth reading.


In [ ]:
#| echo: false
#| warning: false
include("setup.jl")
import MMJMesh.Plots: approximationerror, sampleadaptive

## Plot a 1D mesh

### On a straight line


In [ ]:
m = makemeshoninterval(0, 4, 20)
mplot(m, nodecolor=:green) |> mconf()

One value per node


In [ ]:
mplot(m, -1.1 .+ 2.6 * rand(nentities(m.topology, 0))) |> mconf()

One value per element


In [ ]:
mplot(m, -1.1 .+ 2.2 * rand(nedges(m))) |> mconf()

Two values per element


In [ ]:
mplot(m, -1.1 .+ 2.2 * rand(2, nedges(m))) |> mconf()

### Vertical


In [ ]:
m = makemeshoninterval(π, 3π, 20, t -> [0; t])
mplot(m, -1.1 .+ 3.2 * rand(nedges(m))) |> mconf()

### On a spiral


In [ ]:
m = makemeshoninterval(π, 3π, 20, t -> t * [cos(t); sin(t)])
mplot(m, -1.1 .+ 3.2 * rand(2, nedges(m))) |> mconf()

### Customize plot

Plot customization works like this:


In [ ]:
f, ax = mplot(m, -1.1 .+ 3.2 * rand(2, nedges(m))) |> mconf()
ax.title = "Spiral with reversed vertical axis"
ax.yreversed = true
f

Documentation on plot axis can be found on the [Makie documentation](https://docs.makie.org/stable/reference/blocks/axis) which unfortunately is quite hard to read.

## Plot a 2D mesh

### Quad mesh


In [ ]:
a = 80
m = makemeshonrectangle(9.0, 4.5, 2a, a)
println("Number of nodes is Nn = ", (a + 1) * (2a + 1))
print("Links...")
@time l12 = links(m.topology, 1, 2);

Default color


In [ ]:
mplot(m, edgesvisible=true, edgecolor=:hotpink) |> mconf()

Colors for nodes


In [ ]:
mplot(m, 4.1 * (rand(nentities(m.topology, 0)) .- 0.25)) |> mconf()

Colors for elements


In [ ]:
mplot(m, 4.1 * (rand(nentities(m.topology, 2)) .- 0.25)) |> mconf()

### Triangle mesh


In [ ]:
a = 20
m = makemeshonrectangle(9.0, 4.5, 2a, a, TRIANGLE)
println("Nn = ", (a + 1) * (2a + 1))
print("Links (1, 2):")
@time l12 = links(m.topology, 1, 2);

Default color


In [ ]:
mplot(m, edgesvisible=true) |> mconf()

Colors for nodes


In [ ]:
mplot(m, 4.1 * (rand(nentities(m.topology, 0)) .- 0.25)) |> mconf()

Colors for elements


In [ ]:
mplot(m, 4.1 * (rand(nentities(m.topology, 2)) .- 0.25)) |> mconf()

In [ ]:
m = makemeshoninterval(π, 3π, 20, t -> t * [cos(t); sin(t)])
with_theme(theme_dark()) do
#update_theme!(linecolor=:tomato)
#update_theme!(markercolor=:tomato)

println(theme(:linecolor)[])

#mplot(m)
scatter(coordinates(m))
end

## Plot options


In [ ]:
a = 10
m1 = makemeshonrectangle(4, 2, 2a, a)
mplot(m1, 3 * rand(nfaces(m1)),
    nodesvisible=true, nodecolor=:hotpink, nodesize=12,
    edgesvisible=true, edgecolor=:lightblue, edgelinewidth=3,
    featureedgecolor=:red, featureedgelinewidth=6,
    facecolormap=:bluesreds
) |> mconf()

In [ ]:
m2 = makemeshoninterval(0, 4, 20)
mplot(m2, rand(nnodes(m2)),
    lineplotoutlinesvisible=true,
    edgecolor=:blue, edgelinewidth=10,
    lineplotscale=0.3,
    lineplotoutlinescolor=:hotpink,
    lineplotoutlineslinewidth=2.0,
    lineplotfacescolormap=:bluesreds
) |> mconf(title="Test Plot")

## Adaptive sampling


In [ ]:
#| echo: false
#| warning: false
include("setup.jl")
using IntervalSets
using LinearAlgebra
import CairoMakie as cm
import MMJMesh.Plots: approximationerror, sampleadaptive

### Derivation of quadrature formulae


In [ ]:
X1 = 0:1/4:1
L1 = lagrangepolynomials(X1, 0 .. 1)
W1 = [integrate(L, 0 .. 1) for L in L1]

X2 = [0, 0.253124, 0.4986745, 0.75834, 1]
L2 = lagrangepolynomials(X2, 0 .. 1)
W2 = [integrate(L, 0 .. 1) for L in L2]

function nint(f, a, b)
    w = b - a
    y1 = f.(a .+ w * X1)
    y2 = f.(a .+ w * X2)
    w * dot(W1, y1), w * dot(W2, y2)
end

println(nint(x -> x, 0, 2))
println(nint(sin, 0, pi))
println(nint(sin, pi, 2pi))

### Integral of squared distance between curve and straight line


In [ ]:
#| code-fold: true

function doplot(xy)
    x, y = xy
    f = cm.lines(x, y)
    cm.scatter!(x, y, color=:darkred, markersize=5)
    return f
end

function convergence(f, a, b, n)
    h = 1
    X = X2
    W = W2
    e = zeros(n)

    for i in 1:n
        w = b - a
        x = a .+ w * X
        y = f.(x)
        e[i] = approximationerror(h, X, W, y)
        b = (a + b) / 2
        h /= 2
    end

    return e
end

In [ ]:
n = 5
(1:n, convergence(x -> 100x^3, -0.25, 1.5, n)) |> doplot

In [ ]:
(1:n, convergence(sin, 0, pi, n)) |> doplot

In [ ]:
(1:n, convergence(x -> 1 / x, -1, 1e-5, n)) |> doplot

In [ ]:
n = 20
(1:n, convergence(x -> sin(1 / x), 1e-5, 1e-1, n)) |> doplot

### Test cases


In [ ]:
sampleadaptive(x -> x^100, 0, 1) |> doplot

In [ ]:
sampleadaptive(x -> sin(1.0 / x), 0, 0.05, level=14) |> doplot

In [ ]:
sampleadaptive(x -> 1 / x, -1, 1) |> doplot

In [ ]:
sampleadaptive(x -> (-0.3 + x), -1, 1, ir=true) |> doplot

In [ ]:
sampleadaptive(x -> x^2, -1, 1) |> doplot